In [ ]:
# connect drive :)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import warnings
import numpy as np
import random

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread
import pathlib

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras import models

In [ ]:
img_width=600 ; img_height=600
batch_size=16

In [ ]:
TRAINING_DIR = "/content/drive/MyDrive/New_data_set/1_train/"

train_datagen = ImageDataGenerator(rescale = 1/255.0,
                                   rotation_range=30,
                                   zoom_range=0.4,
                                   shear_range = 0.2,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   horizontal_flip=True,
                                   fill_mode ='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size= batch_size,
                                                    class_mode='categorical',
                                                    target_size=(img_height, img_width))

In [ ]:
VALIDATION_DIR = "/content/drive/MyDrive/New_data_set/validation/"

validation_datagen = ImageDataGenerator(rescale = 1/255.0)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=batch_size,
                                                              class_mode='categorical',
                                                              target_size=(img_height, img_width))

In [ ]:
# callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr=ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience= 10 , verbose=2)

In [ ]:
from keras.applications.efficientnet import EfficientNetB7
conv= EfficientNetB7(weights="imagenet",include_top=False,input_shape=(456 ,456,3))
model=models.Sequential()
model.add(conv)
model.trainable = False

model.add(layers.Flatten())

model.add(layers.Dropout(0.2))

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32 , activation="relu"))


model.add(layers.BatchNormalization())

model.add(layers.Dense(16, activation='softmax'))

In [ ]:
import tensorflow as tf

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    print("Device:", tpu.master())
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("Not connected to a TPU runtime. Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()

In [ ]:
import tensorflow_datasets as tfds
from keras.applications import EfficientNetB7
IMG_SIZE = 600

def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = inputs
    model = EfficientNetB7(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    Dense1 = layers.Dense(512, activation='relu')(x)
    Dense2 = layers.Dense(256, activation='relu')(Dense1)
    Dense3 = layers.Dense(128, activation='relu')(Dense2)
    Dense4 = layers.Dense(64, activation='relu')(Dense3)
    Dense5 = layers.Dense(32, activation='relu')(Dense4)
    outputs = layers.Dense(16, activation="softmax", name="pred")(Dense5)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [ ]:
model = build_model(num_classes=16)
model.summary()

In [ ]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


In [ ]:
with strategy.scope():
    model = build_model(num_classes=16)
    model.summary()

epochs = 8  # @param {type: "slider", min:8, max:80}
hist = model.fit(train_generator, epochs=epochs, validation_data=validation_generator, verbose=2)
plot_hist(hist)

In [ ]:
!pip install efficientnet

In [ ]:
from keras.applications import EfficientNetB7

In [ ]:
conv= EfficientNetB7(weights="imagenet",include_top=False,input_shape=(400 ,400,3))
model=models.Sequential()
model.add(conv)

model.add(layers.Flatten())

model.add(layers.Dropout(0.2))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))

model.add(layers.BatchNormalization())

model.add(layers.Dense(16, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics =['accuracy'])

In [ ]:
history = model.fit(train_generator,
                              epochs= 5,
                              verbose=1,
                              validation_data=validation_generator,
                              callbacks = [reduce_lr]
                              )

In [ ]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc))

fig = plt.figure(figsize=(14,7))
plt.plot(epochs, acc, 'r', label="Training Accuracy")
plt.plot(epochs, val_acc, 'b', label="Validation Accuracy")
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
fig2 = plt.figure(figsize=(14,7))
plt.plot(epochs, loss, 'r', label="Training Loss")
plt.plot(epochs, val_loss, 'b', label="Validation Loss")
plt.legend(loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and validation loss')

In [ ]:
TEST_DIR = "/content/drive/MyDrive/New_data_set/Test/"

Test_datagen = ImageDataGenerator(rescale = 1/255.0)

Test_generator = Test_datagen.flow_from_directory(TEST_DIR,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    target_size=(img_height, img_width))

In [ ]:
Test_generator.class_indices

In [ ]:
class_dict=Test_generator.class_indices
labels= Test_generator.labels
file_names= Test_generator.filenames

In [ ]:
print(len(file_names))

In [ ]:
print(labels)

In [ ]:
test_loss , test_acc = model.evaluate_generator( Test_generator , steps = 50)
print("test accuracy ===>" ,test_acc)
print("test loss =======>",test_loss)